In [60]:
from pathlib import Path
import subprocess
import pandas as pd
import json, pickle,sys,glob
import simplekml, xmltodict
from shapely.geometry import Polygon, Point
import yaml
from mymodules.utils import *
from mymodules.aisight.naqsha import *

In [61]:
# base directory
BASE_DIR = Path().parent.resolve()

with open("config.yaml", "r") as yf:
    config = yaml.safe_load(yf)

# just to add collect script available on path
sys.path.append(Path(config["paths"]["mymodules"]).resolve())

In [62]:
# reading config from yaml
company = config["analysis"]["company"]
country = config["analysis"]["country"]
area = config["analysis"]["area"]

# output folder
# output_folder = Path(config["paths"]["data"]["output"]).resolve()

In [63]:
# update paths section
config['paths'] = replace_placeholders(config['paths'], company, country)

# print(yaml.dump(config, default_flow_style=False))
